In [ ]:
from datetime import date
from itertools import product

import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from sklearn.metrics import mean_absolute_percentage_error, mean_squared_error

from app.model import DispatchConfig
from run_dispatch import run_dispatch

In [ ]:
# DATES = pd.date_range(
#     start="2024-05-01",
#     end="2024-05-31",
#     inclusive="both"
# )
DATES = [
    # pd.Timestamp("2024-01-01"),
    # pd.Timestamp("2024-10-13"),
    # pd.Timestamp("2024-04-28"),
    # pd.Timestamp("2024-03-29"),
    # pd.Timestamp("2024-01-07"),
    # pd.Timestamp("2024-06-09"),
    # pd.Timestamp("2024-09-30"),
    pd.Timestamp("2024-05-30"), #Cluster 1
    pd.Timestamp("2024-10-07"), #Cluster 2
    pd.Timestamp("2024-04-06"), #Cluster 3
    pd.Timestamp("2024-04-26"), #Cluster 4
    pd.Timestamp("2024-04-29"), #Cluster 5
    pd.Timestamp("2024-05-26"), #Cluster 6
    pd.Timestamp("2024-09-30"), #Cluster 7
]
CONFIGS = [
    "preideal",
    # "bess_preideal",
    "bess_preideal_resource",
]

BESS_VALUES = {
    "BESS_1": {
        "MWh_nom": 2000,
        "hours_to_deplete": 2,
        "efficiency": 0.9,
        "min_soc": 0.05,
        "max_soc": 0.98,
        "initial_soc": 0.05,
        "charge_bid": 0,
        "discharge_bid": 0,
    },
}


DERS_YEARS = [None, 2025]


In [ ]:
output_unstacked = pd.DataFrame()
models = []
dispatches_type = []
dates_ = []
DERS_data = []
DERS_names = []
for (date_, dispatch_type, DERS) in product(DATES,CONFIGS, DERS_YEARS):
    BESS = BESS_VALUES if "bess" in dispatch_type.lower() else None
    mpo, model,  pmax_new_resources, expansion_sources = run_dispatch(DispatchConfig(dispatch_type=dispatch_type), DISPATCH_DATE=date_.date(), show_figs=False, BESS=BESS, DERS=DERS)
    dispatches_type.append(dispatch_type)
    models.append(model)
    dates_.append(date_)
    DERS_data.append(pmax_new_resources)
    DERS_names.append(expansion_sources)
    # mpo["dispatch_type"] = dispatch_typex
    output_unstacked = pd.concat([output_unstacked, mpo])
output = output_unstacked.stack().reset_index()
output.columns = ["datetime", "source", "MPO"]
output.drop_duplicates(inplace=True)

In [ ]:
import pyomo.environ as pyo

In [ ]:
fig = go.Figure()
line_type = [t for _ in range(7) for t in ["dot","dot","solid","solid",] ]
for i,(model,dispatch_type, ders_data) in enumerate(zip(models, dispatches_type, DERS_data)):
    if i % 4 == 0:
        fig = go.Figure()
    if demand := dict(model._model.demand.items()):
        demand_df = pd.DataFrame(demand.values(), index=demand.keys(), columns=["demanda"])
    # get BESS charge and discharge
    bess_charge_df = pd.DataFrame(index=model._model.T,)
    bess_charge_df["charge"] = 0
    bess_charge_df["discharge"] = 0
    if "bess" in dispatch_type.lower():
        bess_charge = dict(model._model.bess_charge.items())
        bess_charge = {k: pyo.value(v) for k, v in bess_charge.items()}
        bess_charge_df = pd.DataFrame.from_dict(bess_charge, orient="index", columns=["charge"])
        bess_charge_df.index = pd.Index(data=model._model.T, name="datetime")
        bess_charge_df["discharge"] = [pyo.value(val) for val in model._model.bess_discharge.values()]
        bess_charge_df
    # Add
    if not ders_data.empty:
        ders_data = ders_data.groupby(["datetime"]).agg({"dispo": "sum"})
        demand_df = demand_df.merge(ders_data, how="left", left_index=True, right_index=True)
    demand_df = demand_df.merge(bess_charge_df, how="left", left_index=True, right_index=True)
    if len(demand_df.columns) > 3:
        demand_df["demanda"] = demand_df["demanda"] - demand_df["dispo"]
    demand_df["demanda"] = demand_df["demanda"] - demand_df["discharge"]
    demand_df["demanda"] = demand_df["demanda"] + demand_df["charge"]
    b = "SAEB" if "bess" in dispatch_type.lower() else ""
    d = "DERs" if not ders_data.empty else ""

    name = f"Demanda {'con' if b or d else ''} {b} {'y' if b and d else ''} {d}"
    fig.add_traces([
        go.Scatter(
            x=demand_df.index,
            y=demand_df["demanda"],
            mode="lines",
            name=name,
            line=dict(dash=line_type[i]),
        )
    ])
    fig.update_layout(
        {
            "xaxis_title": "Hora",
            "yaxis_title": "Demanda (MW)",
            "width": 800
        }
    )
    if i in [3, 7, 11, 15, 19, 23, 27, 31]:
        fig.show()

In [ ]:
fig = go.Figure()

for model, dispatch_type, ders_data in zip(models, dispatches_type, DERS_data):
    if demand := dict(model._model.demand.items()):
        demand_df = pd.DataFrame(demand.values(), index=demand.keys(), columns=["demanda"])
    # get BESS charge and discharge
    bess_charge_df = pd.DataFrame(index=model._model.T,)
    bess_charge_df["charge"] = 0
    bess_charge_df["discharge"] = 0
    if "bess" in dispatch_type.lower():
        bess_charge = dict(model._model.bess_charge.items())
        bess_charge = {k: pyo.value(v) for k, v in bess_charge.items()}
        bess_charge_df = pd.DataFrame.from_dict(bess_charge, orient="index", columns=["charge"])
        bess_charge_df.index = pd.Index(data=model._model.T, name="datetime")
        bess_charge_df["discharge"] = [pyo.value(val) for val in model._model.bess_discharge.values()]
        bess_charge_df
    # Add
    if not ders_data.empty:
        ders_data = ders_data.groupby(["datetime"]).agg({"dispo": "sum"})
        demand_df = demand_df.merge(ders_data, how="left", left_index=True, right_index=True)
    demand_df = demand_df.merge(bess_charge_df, how="left", left_index=True, right_index=True)
    if len(demand_df.columns) > 3:
        demand_df["demanda"] = demand_df["demanda"] - demand_df["dispo"]
    demand_df["demanda"] = demand_df["demanda"] - demand_df["discharge"]
    demand_df["demanda"] = demand_df["demanda"] + demand_df["charge"]
    b = "BESS" if "bess" in dispatch_type.lower() else ""
    d = "with DERs" if not ders_data.empty else ""

    name = f"Demanda {b} {d} {date_.date()}"
    fig.add_trace(go.Scatter(
        x=demand_df.index,
        y=demand_df["demanda"],
        mode="lines",
        name=name,
        fill='tozeroy',
        line=dict(color='rgba(0, 100, 80, 1)'),
        fillcolor='rgba(0, 100, 80, 0.2)'  # RGBA color with alpha channel
    ))

fig.update_layout(
    title="Area Chart of Demand",
    xaxis_title="Datetime",
    yaxis_title="Demand",
    showlegend=True
)

fig.show()

In [ ]:
demand_df.diff(axis=1)

In [ ]:
mapper = {
    "MPO preideal XM": "MPO XM",
    "MPO preideal Modelo - DERs None": "MPO Modelo",
    "MPO bess_preideal_resource Modelo - DERs None": "MPO SAEB recurso",
    # "MPO bess_preideal Modelo - DERs None": "MPO Modelo SAEB"
}

output["source"] = output["source"].apply(lambda x: mapper.get(x,x))
output_ = output[output["source"].isin(list(mapper.values()))]

In [ ]:
from plotly.subplots import make_subplots

# Create subplots: 2 columns and 4 rows, with the last row spanning both columns
fig = make_subplots(
    rows=4, cols=2,
    specs=[[{}, {}],
           [{}, {}],
           [{}, {}],
           [{"colspan": 2}, None]],
    subplot_titles=[str(dat_) for dat_ in output_.groupby(output_["datetime"].dt.date).groups.keys()]
)

# Add traces to the subplots
for i, (date, serie) in enumerate(output_.groupby(output_["datetime"].dt.date)):
    row = i // 2 + 1
    col = i % 2 + 1
    showlegend = True if i == 0 else False
    for trace in px.line(serie, x="datetime", y="MPO", color="source", title=str(date)).data:
        trace.showlegend = showlegend
        fig.add_trace(trace, row=row, col=col)

# Update layout
fig.update_layout(
    height=1200, width=800,
    showlegend=True,
    # xaxis=dict(showgrid=True, dtick=3_600_000,),

)
fig.write_html("output.html")
fig.show()

In [ ]:
output_unstacked = output.pivot(index="datetime", columns="source", values="MPO")
output_unstacked

In [ ]:
# Group by days and apply the mean_absolute_percentage_error function
grouped = output_unstacked.groupby(output_unstacked.index.date).apply(
    lambda x: mean_absolute_percentage_error(y_pred=x["MPO Modelo"], y_true=x["MPO XM"])
)

# Convert the result to a DataFrame for better readability
grouped_df = pd.DataFrame(grouped, columns=["MAPE %"])
grouped_df["MAPE %"] = grouped_df["MAPE %"] * 100
grouped_df

In [ ]:
output_unstacked.to_csv("differences_for_month_.csv")